In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from urllib.parse import quote
from tqdm import tqdm

In [2]:
from mainwords import MainWordExtractor
from searcher import Searcher

sentence = "Give me the best results for a tiger running againts a lion"
k = 15

extractor = MainWordExtractor()
ss = Searcher()

search = extractor.extract_mainwords_from_sentence(sentence=sentence)
url= ss.create_search_element(search, top_k = k)
ss.create_article_url(url)

['https://en.wikipedia.org/wiki/Panthera',
 'https://en.wikipedia.org/wiki/Ukrainian collaboration with Nazi Germany',
 'https://en.wikipedia.org/wiki/Tiger',
 'https://en.wikipedia.org/wiki/Smilodon',
 'https://en.wikipedia.org/wiki/Liger',
 'https://en.wikipedia.org/wiki/Animal Face-Off',
 'https://en.wikipedia.org/wiki/MacOS version history',
 'https://en.wikipedia.org/wiki/Cougar',
 'https://en.wikipedia.org/wiki/MacOS',
 'https://en.wikipedia.org/wiki/Panthera hybrid',
 'https://en.wikipedia.org/wiki/Tigon',
 'https://en.wikipedia.org/wiki/Pantherinae',
 'https://en.wikipedia.org/wiki/Ryusuke Taguchi',
 'https://en.wikipedia.org/wiki/Jushin Liger',
 'https://en.wikipedia.org/wiki/Oncilla']

In [2]:
search_url = ['https://en.wikipedia.org/wiki/Panthera',
 'https://en.wikipedia.org/wiki/Ukrainian collaboration with Nazi Germany',
 'https://en.wikipedia.org/wiki/Tiger',
 'https://en.wikipedia.org/wiki/Smilodon',
 'https://en.wikipedia.org/wiki/Liger',
 'https://en.wikipedia.org/wiki/Animal Face-Off',
 'https://en.wikipedia.org/wiki/MacOS version history',
 'https://en.wikipedia.org/wiki/Cougar',
 'https://en.wikipedia.org/wiki/MacOS',
 'https://en.wikipedia.org/wiki/Panthera hybrid',
 'https://en.wikipedia.org/wiki/Tigon',
 'https://en.wikipedia.org/wiki/Pantherinae',
 'https://en.wikipedia.org/wiki/Ryusuke Taguchi',
 'https://en.wikipedia.org/wiki/Jushin Liger',
 'https://en.wikipedia.org/wiki/Oncilla']

In [3]:
from preprocess.cleaner import TextCleaner

cleaner = TextCleaner()

In [4]:
bbdd = cleaner.from_urls_to_dict(search_url)

100%|████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 2502.37it/s]


In [5]:
from preprocess.database import DataBase

database = DataBase()

In [6]:
document_store = database.generate_bbdd(bbdd)

Updating BM25 representation...:   0%|          | 0/115 [00:00<?, ? docs/s]

In [7]:
document_store

In [8]:
from question_answering.extractive_qa import ExtractiveQA

qa = ExtractiveQA(document_store=document_store)

In [9]:
# Values
query  = "When do the  pleistocene extinctions appeared?"
document_hash = "e6b5e715d0495e14409ba9d82d79dc69320617783e1acbac094123b5542f9aa8"


responses = qa.answer(question=query,document_hash=document_hash)

Updating BM25 representation...:   0%|          | 0/5 [00:00<?, ? docs/s]

D:\Archivos de programa\anaconda3\lib\site-packages\torch\utils\data\dataloader.py:645: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x000001D81F2551C0> was reported to be 5 (when accessing len(dataloader)), but 6 samples have been fetched. 
  warnings.warn(warn_msg)
D:\Archivos de programa\anaconda3\lib\site-packages\torch\utils\data\dataloader.py:645: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x000001D81F2551C0> was reported to be 5 (when accessing len(dataloader)), but 7 samples have been fetched. 
  warnings.warn(warn_msg)
D:\Archivos de programa\anaconda3\lib\site-packages\torch\utils\data\dataloader.py:645: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x000001D81F2551C0> was reported to be 5 (when accessing len(dataloader)), but 8 samples have been fetched. 
  warnings.warn(warn_msg)
D:\Archivos de prog

In [10]:
responses

{'answer': '10,000 years ago',
 'context': 'gar population was extirpated during the pleistocene extinctions some 10,000 years ago, when other large mammals, such as smilodon, also disappeared. north '}

In [ ]:
from question_answering.search import SemanticSearch

In [17]:
query = "Give me information about the colonization of the americas"
searcher.search(query=query)

{'documents': [<Document: {'content': 'north america as more people entered cougar habitat and built farms. characteristics intensive hunting following european colonization of the americas and ongoing human development into cougar habitat has caused populations to decline in most parts of its historical range. in particular, the eastern cougar population is considered to be mostly locally extinct in eastern north america since the early 20th century, with the exception of the isolated florida panther subpopulation. distribution and habitat the word cougar is borrowed from the portuguese çuçuarana, via french; it was originally derived from the tupi language. a current form in brazil is suçuarana. in the 17th century, georg marcgrave named it cuguacu ara. marcgrave\'s rendering was reproduced in 1648 by his associate willem piso. cuguacu ara was then adopted by john ray in 1693. in 1774, georges-louis leclerc, comte de buffon converted cuguacu ara to cuguar, which was later modified to

In [8]:
from haystack.nodes import BM25Retriever, SentenceTransformersRanker
from haystack import Pipeline

In [9]:
retriever = BM25Retriever(document_store)
ranker = SentenceTransformersRanker(model_name_or_path="cross-encoder/ms-marco-MiniLM-L-12-v2")

p = Pipeline()
p.add_node(component=retriever, name="BM25Retriever", inputs=["Query"])
p.add_node(component=ranker, name="Ranker", inputs=["BM25Retriever"])

D:\Archivos de programa\anaconda3\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [15]:
p.run(
     query= "Give me information about the colonization of the americas",
     params = {
             "BM25Retriever": {"top_k": 10} ,
             "Ranker": {"top_k": 10}
              }
     )

{'documents': [<Document: {'content': 'north america as more people entered cougar habitat and built farms. characteristics intensive hunting following european colonization of the americas and ongoing human development into cougar habitat has caused populations to decline in most parts of its historical range. in particular, the eastern cougar population is considered to be mostly locally extinct in eastern north america since the early 20th century, with the exception of the isolated florida panther subpopulation. distribution and habitat the word cougar is borrowed from the portuguese çuçuarana, via french; it was originally derived from the tupi language. a current form in brazil is suçuarana. in the 17th century, georg marcgrave named it cuguacu ara. marcgrave\'s rendering was reproduced in 1648 by his associate willem piso. cuguacu ara was then adopted by john ray in 1693. in 1774, georges-louis leclerc, comte de buffon converted cuguacu ara to cuguar, which was later modified to

In [11]:
hashe = bbdd[50]["meta"]["hash"]

In [12]:
hashe

'e6b5e715d0495e14409ba9d82d79dc69320617783e1acbac094123b5542f9aa8'

In [7]:
from question_generation.question_gen import QuestionGenerator

In [8]:
question_gen = QuestionGenerator()

In [9]:
question_gen.generate_questions(texts=bbdd , article_hash=hashe, top_k=5)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.20it/s]


['What is the sister lineage of the tiger?',
 'What is the name of the oldest known panthera species?',
 'How many million years ago did the clouded leopard diverge?',
 'What does the taxonomic assessment in mammal species of the world reflect?',
 'What is the genus neofelis sister to?']

In [14]:
question_gen.generate_questions(texts=bbdd , article_hash=hashe, top_k=2)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:08<00:00,  1.11it/s]


['What is the size of smilodon?', 'What are the earliest felids known from?']

In [15]:
question_gen.generate_questions(texts=bbdd , article_hash=hashe, top_k=2)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.04s/it]


['What is a large cat native to the americas?',
 'What is the most widely distributed large wild terrestrial mammal?']

In [5]:
from haystack.document_stores import InMemoryDocumentStore
from haystack import Document

In [6]:
document_store = InMemoryDocumentStore(use_bm25=True)

In [7]:
document_store.write_documents(bbdd)

Updating BM25 representation...:   0%|          | 0/115 [00:00<?, ? docs/s]

In [9]:
document_store.get_all_documents()